<a href="https://colab.research.google.com/github/achanhon/AdversarialModel/blob/master/vitesse_tirage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import torch
import time

In [ ]:
p = torch.rand(4)*4-2
MAX,a = p.max(0)
Mean = p.mean()
target = 0.9*MAX+0.1*Mean
print(p,MAX,Mean,target)

In [ ]:
T0 = time.time()
out = []
for _ in range(100000):
  if random.random()<0.9:
    out.append(int(a))
  else:
    out.append(int(random.random()*4))
T = time.time()
print(T-T0)
E = [p[i] for i in out]
E = sum(E)/len(E)
print(E)

In [ ]:
soft = torch.nn.functional.softmax(p,dim=0)
softmean = (p*soft).sum()
print(soft,softmean)
assert softmean<target

# seuil * MAX + (1-seuil)*softmean = target
seuil = (target-softmean)/(MAX-softmean)

T0 = time.time()
out = []
for _ in range(100000):
  if random.random()<seuil:
    out.append(int(a))
  else:
    out.append(int(torch.multinomial(soft, 1).item()))
T = time.time()
print(T-T0)
E = [p[i] for i in out]
E = sum(E)/len(E)
print(E)

In [ ]:
import torch
import random

class Net(torch.nn.Module):
  def __init__(self):
    super(Net,self).__init__()

    self.f1 = torch.nn.Linear(4,32)
    self.f2 = torch.nn.Linear(32,1)

  def forward(self,x):
    return self.f2(torch.nn.functional.relu(self.f1(x)))

R = torch.rand(10)*10-5
X = torch.rand(10,4)

print("option 1")
net = Net()
optimizer = torch.optim.SGD(net.parameters(),lr=0.0005)
log=[]
for i in range(10000):
  j = int(random.random()*10)
  if j==9:
    loss = (net(X[j])-R[j])**2
  else:
    loss = (net(X[j])-net(X[j+1])-R[j])**2

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i%100==99:
    print(i,sum(log)/100)
    log = []
  else:
    log.append(float(loss))

print("option 2")
net = Net()
optimizer = torch.optim.SGD(net.parameters(),lr=0.0005)
log = []
for i in range(10000):
  j = int(random.random()*10)
  if j==9:
    loss = (net(X[j])-R[j])**2
  else:
    with torch.no_grad():
      NEXT = net(X[j+1])
    loss = (net(X[j])-NEXT-R[j])**2

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i%100==99:
    print(i,sum(log)/100)
    log = []
  else:
    log.append(float(loss))

sans "max", il n'y a aucune raison de torch no_grad !!! mais avec, apparemment si...